In [1]:
# Importar las bibliotecas necesarias
import sys
import os
import pandas as pd
from sklearn.exceptions import NotFittedError
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#Configuracion de seaborn
sns.set_theme(style='whitegrid', context='paper', palette='muted')

# Agregar el directorio de scripts al path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'scripts'))

# Importar el logger personalizado
from logger import CustomLogger

# Inicializar el logger
logger = CustomLogger(developer='David')
app_logger = logger.get_logger('app')
errors_logger = logger.get_logger('errors')

try:
    # Cargar los datos de entrenamiento
    train_data = pd.read_csv('../data/train.csv')
    app_logger.info("Conjunto de datos de entrenamiento cargado exitosamente.")
except FileNotFoundError:
    errors_logger.error("No se pudo encontrar el archivo de datos de entrenamiento.")
    raise
except Exception as e:
    errors_logger.error(f"Error al cargar los datos de entrenamiento: {str(e)}")
    raise

# Cargamos test.csv
try:
    test_data = pd.read_csv('../data/test.csv')
    app_logger.info("Conjunto de datos de test cargado exitosamente.")
except FileNotFoundError:
    errors_logger.error("No se pudo encontrar el archivo de datos de test.")
    raise

# Mostrar las primeras filas del conjunto de datos
train_data.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
# Valores faltantes
print("Valores faltantes por columna:")
valores_faltantes = train_data.isnull().sum()
porcentaje_faltantes = 100 * train_data.isnull().sum() / len(train_data)
tabla_faltantes = pd.concat([valores_faltantes, porcentaje_faltantes], axis=1, keys=['Total', 'Porcentaje'])
print(tabla_faltantes[tabla_faltantes['Total'] > 0].sort_values('Total', ascending=False))

# Filas duplicadas
filas_duplicadas = train_data.duplicated().sum()
print(f"\nNúmero de filas duplicadas: {filas_duplicadas}")

# Registrar en el log
if valores_faltantes.sum() > 0:
    app_logger.info(f"Se encontraron {valores_faltantes.sum()} valores faltantes en total.")
else:
    app_logger.info("No se encontraron valores faltantes en el conjunto de datos.")

if filas_duplicadas > 0:
    app_logger.warning(f"Se encontraron {filas_duplicadas} filas duplicadas en el conjunto de datos.")
else:
    app_logger.info("No se encontraron filas duplicadas en el conjunto de datos.")


Valores faltantes por columna:
              Total  Porcentaje
PoolQC         1453   99.520548
MiscFeature    1406   96.301370
Alley          1369   93.767123
Fence          1179   80.753425
FireplaceQu     690   47.260274
LotFrontage     259   17.739726
GarageType       81    5.547945
GarageYrBlt      81    5.547945
GarageFinish     81    5.547945
GarageQual       81    5.547945
GarageCond       81    5.547945
BsmtExposure     38    2.602740
BsmtFinType2     38    2.602740
BsmtFinType1     37    2.534247
BsmtCond         37    2.534247
BsmtQual         37    2.534247
MasVnrArea        8    0.547945
MasVnrType        8    0.547945
Electrical        1    0.068493

Número de filas duplicadas: 0


In [3]:
# Valores faltantes en el conjunto de test
print("Valores faltantes por columna en el conjunto de test:")
valores_faltantes_test = test_data.isnull().sum()
porcentaje_faltantes_test = 100 * test_data.isnull().sum() / len(test_data)
tabla_faltantes_test = pd.concat([valores_faltantes_test, porcentaje_faltantes_test], axis=1, keys=['Total', 'Porcentaje'])
print(tabla_faltantes_test[tabla_faltantes_test['Total'] > 0].sort_values('Total', ascending=False))

# Filas duplicadas en el conjunto de test
filas_duplicadas_test = test_data.duplicated().sum()
print(f"\nNúmero de filas duplicadas en el conjunto de test: {filas_duplicadas_test}")

# Registrar en el log para el conjunto de test
if valores_faltantes_test.sum() > 0:
    app_logger.info(f"Se encontraron {valores_faltantes_test.sum()} valores faltantes en total en el conjunto de test.")
else:
    app_logger.info("No se encontraron valores faltantes en el conjunto de test.")

if filas_duplicadas_test > 0:
    app_logger.warning(f"Se encontraron {filas_duplicadas_test} filas duplicadas en el conjunto de test.")
else:
    app_logger.info("No se encontraron filas duplicadas en el conjunto de test.")


Valores faltantes por columna en el conjunto de test:
              Total  Porcentaje
PoolQC         1456   99.794380
MiscFeature    1408   96.504455
Alley          1352   92.666210
Fence          1169   80.123372
FireplaceQu     730   50.034270
LotFrontage     227   15.558602
GarageCond       78    5.346127
GarageYrBlt      78    5.346127
GarageQual       78    5.346127
GarageFinish     78    5.346127
GarageType       76    5.209047
BsmtCond         45    3.084304
BsmtExposure     44    3.015764
BsmtQual         44    3.015764
BsmtFinType1     42    2.878684
BsmtFinType2     42    2.878684
MasVnrType       16    1.096642
MasVnrArea       15    1.028101
MSZoning          4    0.274160
BsmtFullBath      2    0.137080
BsmtHalfBath      2    0.137080
Functional        2    0.137080
Utilities         2    0.137080
GarageCars        1    0.068540
GarageArea        1    0.068540
TotalBsmtSF       1    0.068540
KitchenQual       1    0.068540
BsmtUnfSF         1    0.068540
BsmtFinSF2        

## Missing Columns on Train
- Pool Quality Col has 99,52% missing values, but we still have Pool Area wich i think is more important than PoolQc
- MiscFeature Col has 96.30% missing values. We also have MiscVal that represents the value of Misc Feature.
    - Each Misc Feature has a unique value or has multiple values?? If each MiscFeature has only one unique value (univoque relationship) we can drop MiscFeature and leave Misc Val because we wont lose info, otherwise, drop both. Even if MiscVal has no NaNs, using it on its own may not be the best for the model to gather relationships.
- Alley has 93% missing values, no other col is related to this one so we can directly drop it.


## Verificar si la relación es unívoca entre MiscFeature y MiscVal

In [4]:
# Crear un DataFrame con MiscFeature y MiscVal
misc_df = train_data[['MiscFeature', 'MiscVal']]

# Agrupar por MiscFeature y contar los valores únicos de MiscVal
relacion_misc = misc_df.groupby('MiscFeature')['MiscVal'].nunique().reset_index()
relacion_misc.columns = ['MiscFeature', 'Valores_Unicos_MiscVal']

print("Relación entre MiscFeature y MiscVal:")
print(relacion_misc)

# Verificar si cada MiscFeature tiene un único valor de MiscVal
es_univoca = (relacion_misc['Valores_Unicos_MiscVal'] == 1).all()

if es_univoca:
    print("\nLa relación entre MiscFeature y MiscVal es unívoca.")
    app_logger.info("La relación entre MiscFeature y MiscVal es unívoca. Se puede considerar dejar una columna.")
else:
    print("\nLa relación entre MiscFeature y MiscVal no es unívoca.")
    app_logger.info("La relación entre MiscFeature y MiscVal no es unívoca. Se recomienda eliminar ambas columnas.")

# Mostrar ejemplos de MiscFeature con múltiples valores de MiscVal
if not es_univoca:
    print("\nEjemplos de MiscFeature con múltiples valores de MiscVal:")
    ejemplos_multiples = relacion_misc[relacion_misc['Valores_Unicos_MiscVal'] > 1]
    for _, row in ejemplos_multiples.iterrows():
        feature = row['MiscFeature']
        valores = misc_df[misc_df['MiscFeature'] == feature]['MiscVal'].unique()
        print(f"MiscFeature: {feature}")
        print(f"Valores de MiscVal: {valores}\n")


Relación entre MiscFeature y MiscVal:
  MiscFeature  Valores_Unicos_MiscVal
0        Gar2                       2
1        Othr                       2
2        Shed                      18
3        TenC                       1

La relación entre MiscFeature y MiscVal no es unívoca.

Ejemplos de MiscFeature con múltiples valores de MiscVal:
MiscFeature: Gar2
Valores de MiscVal: [15500  8300]

MiscFeature: Othr
Valores de MiscVal: [3500    0]

MiscFeature: Shed
Valores de MiscVal: [ 700  350  500  400  480  450 1200  800 2000  600 1300   54  620  560
 1400    0 1150 2500]



In [5]:
# Dado que MiscVal y MiscFeature no son univocamente relacionados, se eliminan ambas columnas puesto que estan relacionadas y no se tienen datos de una y la otra son casi todo 0
train_data = train_data.drop(columns=['MiscFeature', 'MiscVal'])
test_data = test_data.drop(columns=['MiscFeature', 'MiscVal'])

In [6]:
# Filtrar columnas con más del 90% de valores faltantes
umbral_faltantes = 0.9
columnas_a_eliminar = tabla_faltantes[tabla_faltantes['Porcentaje'] > 90].index
train_data_filtrado = train_data.drop(columns=columnas_a_eliminar.drop(['MiscFeature']))

# Registrar en el log
if len(columnas_a_eliminar) > 0:
    app_logger.info(f"Se eliminaron {len(columnas_a_eliminar)} columnas con más del 90% de valores faltantes: {', '.join(columnas_a_eliminar)}")
    print(f"Columnas eliminadas: {', '.join(columnas_a_eliminar.drop(['MiscFeature']))}")
else:
    app_logger.info("No se encontraron columnas con más del 90% de valores faltantes.")
    print("No se encontraron columnas con más del 90% de valores faltantes.")

# Mostrar la forma del nuevo conjunto de datos
print(f"\nForma del conjunto de datos original: {train_data.shape}")
print(f"Forma del conjunto de datos filtrado: {train_data_filtrado.shape}")


Columnas eliminadas: Alley, PoolQC

Forma del conjunto de datos original: (1460, 79)
Forma del conjunto de datos filtrado: (1460, 77)


In [7]:
# Importar dtale
import dtale

# Crear una instancia de dtale con los datos filtrados
d = dtale.show(train_data_filtrado)

# Mostrar el enlace para acceder a la interfaz de dtale
print("Se ha generado un análisis interactivo con dtale.")
print(f"Por favor, acceda a la siguiente URL para explorar los datos: {d._url}")

# Registrar en el log
app_logger.info("Se ha generado un análisis interactivo utilizando dtale.")


c:\Users\david\anaconda3\envs\david_pcc\lib\site-packages\dtale\views.py:798: FutureWarning:

['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'Fence'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.

2024-07-22 03:26:26,627 - INFO     - Se ha generado un análisis interactivo utilizando dtale.


Se ha generado un análisis interactivo con dtale.
Por favor, acceda a la siguiente URL para explorar los datos: http://PortatilDavid:40000


## Analysis D-Tale
- We found important predictive power with the following features ``FullBath_quantile, FullBath, GrLiveArea, GrLiveArea_power, GrLiveArea_quantile,GrLiveArea_robust, GarageCars, OverallQual``
- Maybe add a feature that stratifies if the house has garage or not (Never Mind ``GarageCond`` and ``GarageQual`` already do).
- 81 houses have 0 car capacity in garage and 0 sqft but not NA specified on ``GarageCond``, ``GarageQual``, ``GarageType`` and ``GarageFinish``, drop rows with 0 or impute them. If a house has 0 for ``GarageCars`` and ``GarageArea``, the ``GarageType``, ``GarageCond``, ``GarageQual``, and ``GarageFinish`` should be NA?.
- ``GarageQual`` and ``GarageCond`` are duplicates.
- The features ``Fireplaces`` and ``FireplaceQu`` are both NaN or 0 for the same row, so imputation should be done simultaneosly maintaining their possible relationship, imputing the mean and most frecuent values independently could potentially introduce noise as the pairing could possibly be random.

- Most of the numerical variables where Non-normal, with log relationship with sales price (target).

    ### Low Variance Features
    - Dtale showed the following features having low variance:
        - ``BsmtFinSF2`` : 88.56% are 0s, related to this feature we have ``BsmtFinType2`` with 88.33% of rows with value 'Unf' which means 'unfinished '. This could mean that as it is unfinshed the sqft on ``BsmtFinSF2`` are not taken into account so the value will be 0?. Coincidence percentaje between ``BsmtFinType2 == 'Unf' and BsmtFinSF2 == 0``: 97.14%
        - ``LowQualFinSF``: 98.22% are 0s. This feature represents 'Low quality finished square feet (all floors)', i dont really know if we could impute this values as the meaning of the variable is not deterministic or easy to define because it represent the sqft of LowQuality finished sqft of all floors, but there isnt a feature defining wich houses are LowQuality or finished. 
            - Maybe we could create a feature called ``Total_sqft`` and/or Mean between all floors per house, etc... 
        - ``KitchenAbvGr``: From possible values {0,1,2,3} 95.34% are 1s. These feature represents the number of kitchens above grade. There is also the ``TotRmsAbvGrd`` feature measuring the total rooms above grade and the feature ``KitchenQual`` wich indicates the quality of the kitchens. Due to this i think we could drop this feature and leave the other two, as the model will probably discover the relationships.
        - ``PoolArea``: 99.52% values are 0s. The related feature was ``PoolQC`` representing the pool quality but it had 99.79% of NaNs values so i think we can drop both.
        - ``EnclosedPorch``: 85.75% of 0s.
        - ``3SsnPorch``: 98.36% of 0s.
        - ``ScreenPorch``: 92.05% of 0s.
        - Out of this 3 features realted with sqft of different types of Porch the one that has the most sens on maintaining on the dataset is ``EnclosedPorch`` that is de opposite to ``OpenPorch`` (that measures the open porch sqft) and its not that much full of 0s as the other two. This could be okay with te model as it can learn that if we have some value on ``EnclosedPorch``, the house has an enclosed porch, and the same for ``OpenPorch``. The other 2 features are practically all 0s and not much value can be obtained. I think the 2 important features are the ones i mentioned, we could do some new features to encode this info of having 1 porch or another or both.
        

In [8]:
# Comparar YearRemodAdd con YearBuilt para identificar casas remodeladas
casas_remodeladas = train_data_filtrado[train_data_filtrado['YearRemodAdd'] != train_data_filtrado['YearBuilt']]
casas_no_remodeladas = train_data_filtrado[train_data_filtrado['YearRemodAdd'] == train_data_filtrado['YearBuilt']]

# Calcular el número de casas remodeladas y no remodeladas
num_remodeladas = len(casas_remodeladas)
num_no_remodeladas = len(casas_no_remodeladas)

# Imprimir los resultados
print(f"Número de casas remodeladas: {num_remodeladas}")
print(f"Número de casas no remodeladas: {num_no_remodeladas}")
print(f"Porcentaje de casas remodeladas: {(num_remodeladas / len(train_data_filtrado)) * 100:.2f}%")

# Registrar en el log
app_logger.info(f"Se identificaron {num_remodeladas} casas remodeladas y {num_no_remodeladas} casas no remodeladas.")


2024-07-22 03:26:26,689 - INFO     - Se identificaron 696 casas remodeladas y 764 casas no remodeladas.


Número de casas remodeladas: 696
Número de casas no remodeladas: 764
Porcentaje de casas remodeladas: 47.67%


In [9]:

# Comparar YearBuilt con GarageYrBlt
coinciden = train_data_filtrado['YearBuilt'] == train_data_filtrado['GarageYrBlt']
diferentes = train_data_filtrado['YearBuilt'] != train_data_filtrado['GarageYrBlt']

# Calcular porcentajes
porcentaje_coinciden = (coinciden.sum() / len(train_data_filtrado)) * 100
porcentaje_diferentes = (diferentes.sum() / len(train_data_filtrado)) * 100

print(f"Porcentaje de casas donde YearBuilt coincide con GarageYrBlt: {porcentaje_coinciden:.2f}%")

# Analizar cuántas de las que no coinciden con YearBuilt coinciden con YearRemodAdd
coinciden_con_remod = train_data_filtrado[diferentes]['GarageYrBlt'] == train_data_filtrado[diferentes]['YearRemodAdd']
porcentaje_coinciden_remod = (coinciden_con_remod.sum() / diferentes.sum()) * 100

print(f"Del {porcentaje_diferentes:.2f}% que no coincide con YearBuilt, {porcentaje_coinciden_remod:.2f}% coincide con YearRemodAdd")
print(f"Porcentaje de casas donde YearBuilt es diferente de GarageYrBlt: {porcentaje_diferentes:.2f}%")

# Calcular el porcentaje que no coincide ni con YearBuilt ni con YearRemodAdd
no_coincide_ninguno = (~coinciden) & (~coinciden_con_remod)
porcentaje_no_coincide_ninguno = (no_coincide_ninguno.sum() / len(train_data_filtrado)) * 100

print(f"Porcentaje de casas donde GarageYrBlt no coincide ni con YearBuilt ni con YearRemodAdd: {porcentaje_no_coincide_ninguno:.2f}%")

# Evaluar la proximidad de GarageYrBlt con YearBuilt y YearRemodAdd de las que no coinciden con YearBuilt ni con YearRemodAdd
no_coincide_ninguno = train_data_filtrado[no_coincide_ninguno].copy()
no_coincide_ninguno['diff_with_yearbuilt'] = abs(no_coincide_ninguno['GarageYrBlt'] - no_coincide_ninguno['YearBuilt'])
no_coincide_ninguno['diff_with_yearremodadd'] = abs(no_coincide_ninguno['GarageYrBlt'] - no_coincide_ninguno['YearRemodAdd'])

# Determinar proximidad dentro de 10 años
proxima_a_yearbuilt = no_coincide_ninguno[no_coincide_ninguno['diff_with_yearbuilt'] <= 7]
proxima_a_yearremodadd = no_coincide_ninguno[no_coincide_ninguno['diff_with_yearremodadd'] <= 7]
mas_de_10_anos = no_coincide_ninguno[(no_coincide_ninguno['diff_with_yearbuilt'] > 7) & (no_coincide_ninguno['diff_with_yearremodadd'] > 7)]

# Calcular porcentajes
porcentaje_proxima_a_yearbuilt = (len(proxima_a_yearbuilt) / len(no_coincide_ninguno)) * 100
porcentaje_proxima_a_yearremodadd = (len(proxima_a_yearremodadd) / len(no_coincide_ninguno)) * 100
porcentaje_mas_de_10_anos = (len(mas_de_10_anos) / len(no_coincide_ninguno)) * 100
# Calcular el porcentaje que representan las casas con más de 10 años de diferencia respecto al total
porcentaje_mas_10_anos_total = (len(mas_de_10_anos) / len(train_data_filtrado)) * 100

print(f"Porcentaje de casas donde GarageYrBlt está a más de 10 años de YearBuilt y YearRemodAdd respecto al total: {porcentaje_mas_10_anos_total:.2f}%")

# Registrar en el log
app_logger.info(f"El {porcentaje_mas_10_anos_total:.2f}% del total de casas tienen GarageYrBlt a más de 10 años de YearBuilt y YearRemodAdd.")

print(f"Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a menos de 10 años de YearBuilt: {porcentaje_proxima_a_yearbuilt:.2f}%")
print(f"Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a menos de 10 años de YearRemodAdd: {porcentaje_proxima_a_yearremodadd:.2f}%")
print(f"Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a más de 10 años de ambas fechas: {porcentaje_mas_de_10_anos:.2f}%")

# Registrar en el log
import logging
logging.basicConfig(level=logging.INFO)
app_logger = logging.getLogger(__name__)

app_logger.info(f"El {porcentaje_no_coincide_ninguno:.2f}% de las casas tienen GarageYrBlt que no coincide ni con YearBuilt ni con YearRemodAdd.")
app_logger.info(f"Porcentaje de casas donde GarageYrBlt está a menos de 10 años de YearBuilt: {porcentaje_proxima_a_yearbuilt:.2f}%")
app_logger.info(f"Porcentaje de casas donde GarageYrBlt está a menos de 10 años de YearRemodAdd: {porcentaje_proxima_a_yearremodadd:.2f}%")
app_logger.info(f"Porcentaje de casas donde GarageYrBlt está a más de 10 años de ambas fechas: {porcentaje_mas_de_10_anos:.2f}%")

# Calcular tiempos medios y extremos
tiempo_medio_construccion = (train_data_filtrado['GarageYrBlt'] - train_data_filtrado['YearBuilt']).mean()
tiempo_minimo_construccion = (train_data_filtrado['GarageYrBlt'] - train_data_filtrado['YearBuilt']).min()
tiempo_maximo_construccion = (train_data_filtrado['GarageYrBlt'] - train_data_filtrado['YearBuilt']).max()

tiempo_medio_remodelacion = (train_data_filtrado['YearRemodAdd'] - train_data_filtrado['GarageYrBlt']).mean()
tiempo_minimo_remodelacion = (train_data_filtrado['YearRemodAdd'] - train_data_filtrado['GarageYrBlt']).min()
tiempo_maximo_remodelacion = (train_data_filtrado['YearRemodAdd'] - train_data_filtrado['GarageYrBlt']).max()

tiempo_medio_hasta_remodelacion = (train_data_filtrado['YearRemodAdd'] - train_data_filtrado['YearBuilt']).mean()
tiempo_minimo_hasta_remodelacion = (train_data_filtrado['YearRemodAdd'] - train_data_filtrado['YearBuilt']).min()
tiempo_maximo_hasta_remodelacion = (train_data_filtrado['YearRemodAdd'] - train_data_filtrado['YearBuilt']).max()

print(f"Tiempo medio entre GarageYrBlt y YearBuilt: {tiempo_medio_construccion:.2f} años")
print(f"Tiempo medio entre GarageYrBlt y YearRemodAdd: {tiempo_medio_remodelacion:.2f} años")
print(f"Tiempo mínimo entre GarageYrBlt y YearBuilt: {tiempo_minimo_construccion:.2f} años")
print(f"Tiempo máximo entre GarageYrBlt y YearBuilt: {tiempo_maximo_construccion:.2f} años")
print(f"Tiempo mínimo entre GarageYrBlt y YearRemodAdd: {tiempo_minimo_remodelacion:.2f} años")
print(f"Tiempo máximo entre GarageYrBlt y YearRemodAdd: {tiempo_maximo_remodelacion:.2f} años")
print(f"Tiempo medio entre YearBuilt y YearRemodAdd: {tiempo_medio_hasta_remodelacion:.2f} años")
print(f"Tiempo mínimo entre YearBuilt y YearRemodAdd: {tiempo_minimo_hasta_remodelacion:.2f} años")
print(f"Tiempo máximo entre YearBuilt y YearRemodAdd: {tiempo_maximo_hasta_remodelacion:.2f} años")

# Registrar en el log
app_logger.info(f"Análisis de YearBuilt vs GarageYrBlt completado. "
                f"Coinciden: {porcentaje_coinciden:.2f}%, Diferentes: {porcentaje_diferentes:.2f}%")
app_logger.info(f"Para casas con garage sin terminar, tiempo medio entre construcción: "
                f"{tiempo_medio_construccion:.2f} años, tiempo medio hasta remodelación: "
                f"{tiempo_medio_remodelacion:.2f} años")


2024-07-22 03:26:26,721 - INFO     - El 9.45% del total de casas tienen GarageYrBlt a más de 10 años de YearBuilt y YearRemodAdd.
2024-07-22 03:26:26,722 - INFO     - El 21.71% de las casas tienen GarageYrBlt que no coincide ni con YearBuilt ni con YearRemodAdd.
2024-07-22 03:26:26,723 - INFO     - Porcentaje de casas donde GarageYrBlt está a menos de 10 años de YearBuilt: 22.08%
2024-07-22 03:26:26,723 - INFO     - Porcentaje de casas donde GarageYrBlt está a menos de 10 años de YearRemodAdd: 23.03%
2024-07-22 03:26:26,724 - INFO     - Porcentaje de casas donde GarageYrBlt está a más de 10 años de ambas fechas: 43.53%
2024-07-22 03:26:26,727 - INFO     - Análisis de YearBuilt vs GarageYrBlt completado. Coinciden: 74.59%, Diferentes: 25.41%
2024-07-22 03:26:26,727 - INFO     - Para casas con garage sin terminar, tiempo medio entre construcción: 5.55 años, tiempo medio hasta remodelación: 6.93 años


Porcentaje de casas donde YearBuilt coincide con GarageYrBlt: 74.59%
Del 25.41% que no coincide con YearBuilt, 14.56% coincide con YearRemodAdd
Porcentaje de casas donde YearBuilt es diferente de GarageYrBlt: 25.41%
Porcentaje de casas donde GarageYrBlt no coincide ni con YearBuilt ni con YearRemodAdd: 21.71%
Porcentaje de casas donde GarageYrBlt está a más de 10 años de YearBuilt y YearRemodAdd respecto al total: 9.45%
Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a menos de 10 años de YearBuilt: 22.08%
Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a menos de 10 años de YearRemodAdd: 23.03%
Porcentaje de casas donde GarageYrBlt no coincide con YearBuilt ni con YearRemodAdd y está a más de 10 años de ambas fechas: 43.53%
Tiempo medio entre GarageYrBlt y YearBuilt: 5.55 años
Tiempo medio entre GarageYrBlt y YearRemodAdd: 6.93 años
Tiempo mínimo entre GarageYrBlt y YearBuilt: -10.00 años
Tiempo máxi

In [10]:
# Calcular el porcentaje de coincidencia entre BsmtFinType2 == 'Unf' y BsmtFinSF2 == 0
coincidencias = ((train_data_filtrado['BsmtFinType2'] == 'Unf') & (train_data_filtrado['BsmtFinSF2'] == 0)).sum()
total_casos_0 = (train_data_filtrado['BsmtFinSF2']==0).sum()
porcentaje_coincidencia = (coincidencias / total_casos_0) * 100

# Imprimir el resultado
print(f"Porcentaje de coincidencia entre BsmtFinType2 == 'Unf' y BsmtFinSF2 == 0: {porcentaje_coincidencia:.2f}%")

# Registrar en el log
app_logger.info(f"El {porcentaje_coincidencia:.2f}% de los casos tienen BsmtFinType2 == 'Unf' y BsmtFinSF2 == 0 simultáneamente.")

# Calcular casos donde no coinciden
no_coincidencias = ((train_data_filtrado['BsmtFinType2'] == 'Unf') & (train_data_filtrado['BsmtFinSF2'] != 0)).sum()
total_casos_no_0 = len(train_data_filtrado['BsmtFinSF2']!=0)
porcentaje_no_coincidencia = (no_coincidencias / total_casos_no_0) * 100

print(f"Porcentaje de casos donde BsmtFinType2 == 'Unf' pero BsmtFinSF2 != 0: {porcentaje_no_coincidencia:.2f}%")
app_logger.info(f"El {porcentaje_no_coincidencia:.2f}% de los casos tienen BsmtFinType2 == 'Unf' pero BsmtFinSF2 != 0.")

# Analizar las combinaciones con BsmtFinSF2 == 0
combinaciones_cero = train_data_filtrado[train_data_filtrado['BsmtFinSF2'] == 0]['BsmtFinType2'].value_counts()
porcentaje_combinaciones = (combinaciones_cero / len(train_data_filtrado)) * 100

print("Combinaciones con BsmtFinSF2 == 0:")
for tipo, conteo in combinaciones_cero.items():
    porcentaje = porcentaje_combinaciones[tipo]
    print(f"BsmtFinType2 = '{tipo}': {conteo} casos ({porcentaje:.2f}%)")

# Registrar en el log
app_logger.info("Análisis de combinaciones con BsmtFinSF2 == 0:")
for tipo, porcentaje in porcentaje_combinaciones.items():
    app_logger.info(f"BsmtFinType2 = '{tipo}': {porcentaje:.2f}% de los casos")

# Calcular el porcentaje total de casos con BsmtFinSF2 == 0
porcentaje_total_cero = (train_data_filtrado['BsmtFinSF2'] == 0).mean() * 100
print(f"\nPorcentaje total de casos con BsmtFinSF2 == 0: {porcentaje_total_cero:.2f}%")
app_logger.info(f"El {porcentaje_total_cero:.2f}% de los casos tienen BsmtFinSF2 == 0")



2024-07-22 03:26:26,750 - INFO     - El 97.14% de los casos tienen BsmtFinType2 == 'Unf' y BsmtFinSF2 == 0 simultáneamente.
2024-07-22 03:26:26,752 - INFO     - El 0.00% de los casos tienen BsmtFinType2 == 'Unf' pero BsmtFinSF2 != 0.
2024-07-22 03:26:26,755 - INFO     - Análisis de combinaciones con BsmtFinSF2 == 0:
2024-07-22 03:26:26,756 - INFO     - BsmtFinType2 = 'Unf': 86.03% de los casos
2024-07-22 03:26:26,757 - INFO     - El 88.56% de los casos tienen BsmtFinSF2 == 0


Porcentaje de coincidencia entre BsmtFinType2 == 'Unf' y BsmtFinSF2 == 0: 97.14%
Porcentaje de casos donde BsmtFinType2 == 'Unf' pero BsmtFinSF2 != 0: 0.00%
Combinaciones con BsmtFinSF2 == 0:
BsmtFinType2 = 'Unf': 1256 casos (86.03%)

Porcentaje total de casos con BsmtFinSF2 == 0: 88.56%


## XiCorr Coeficiente
El coeficiente Xicorr (Xi-correlation coefficient) es una medida de asociación entre dos variables que puede ser utilizada como alternativa al coeficiente de correlación de Pearson, especialmente cuando las variables no tienen una relación lineal.

### Definición del Coeficiente Xicorr

El coeficiente Xicorr se basa en la idea de las "permutaciones locales". Se enfoca en la desviación de las permutaciones locales de una relación ideal, y puede ser más robusto frente a distribuciones no normales y relaciones no lineales.

#### 1. Supongamos que tenemos dos variables $X$ e $Y$ con $n$ observaciones cada una.

$$ X = (X_1, X_2, \ldots, X_n) $$
$$ Y = (Y_1, Y_2, \ldots, Y_n) $$


#### 2. Ordenamos ambas variables en orden ascendente.

$$ X_{(1)}, X_{(2)}, \ldots, X_{(n)} $$
$$ Y_{(1)}, Y_{(2)}, \ldots, Y_{(n)} $$

#### 3. Calculamos las posiciones de los valores originales en los vectores ordenados. Denotamos las posiciones de $X_i$ en $X_{(i)}$ como $P_X(i)$ y de $Y_i$ en $Y_{(i)}$ como $P_Y(i)$.

$$ P_X(i) = \text{posición de } X_i \text{ en } X_{(i)} $$
$$ P_Y(i) = \text{posición de } Y_i \text{ en } Y_{(i)} $$

#### 4. Definimos las permutaciones locales $ \pi_X $ y $ \pi_Y $ de $X$ e $Y$ respectivamente, que representan cómo se permutan los valores cuando se ordenan.

#### 5. Calculamos la desviación de estas permutaciones locales de una relación ideal. Esta desviación se mide a través de una función de distancia. Una forma común de definir esta distancia es usar la distancia de Kendall ($ \tau $).

$$ \tau(\pi_X, \pi_Y) = \text{Número de discordancias entre } \pi_X \text{ y } \pi_Y $$

#### 6. Normalizamos esta distancia para obtener el coeficiente Xicorr. La normalización se hace para que el coeficiente esté en el rango [-1, 1], similar a los coeficientes de correlación tradicionales.

$$ \text{Xicorr}(X, Y) = 1 - \frac{2 \tau(\pi_X, \pi_Y)}{n(n-1)/2} $$

Aquí, $ n(n-1)/2 $ es el número total de pares posibles en el conjunto de datos.

### Interpretación del Coeficiente Xicorr

- **Xicorr = 1**: Indica una correlación perfecta y positiva entre las variables $X$ e $Y$.
- **Xicorr = -1**: Indica una correlación perfecta y negativa entre las variables $X$ e $Y$.
- **Xicorr = 0**: Indica que no hay correlación entre las variables $X$ e $Y$.

### Pasos Intuitivos del Cálculo del Coeficiente Xicorr

1. **Ordenación de las Variables**:
   - Imagina que tienes dos listas de números, una para cada variable, \(X\) e \(Y\). Primero, ordenas cada lista de menor a mayor.
   - Por ejemplo, si \(X = [4, 1, 3]\), lo ordenas como \(X_{ordenado} = [1, 3, 4]\). Lo mismo haces con \(Y\).

2. **Rastreo de las Posiciones Originales**:
   - Luego, haces un seguimiento de las posiciones originales de los elementos en las listas ordenadas. Es decir, determinas dónde estaba originalmente cada valor en la lista desordenada.
   - Por ejemplo, si \(X = [4, 1, 3]\) y \(X_{ordenado} = [1, 3, 4]\), el valor 1 estaba en la posición 2 originalmente, el 3 en la posición 3 y el 4 en la posición 1.

3. **Comparación de las Permutaciones**:
   - Ahora, haces lo mismo para la variable \(Y\). Una vez que tienes las posiciones originales de ambos conjuntos de datos ordenados, puedes compararlas.
   - Imagina que tienes las posiciones originales de \(X\) como \(\pi_X\) y las posiciones originales de \(Y\) como \(\pi_Y\).

4. **Medición de la Discordancia**:
   - Comparas estas permutaciones (\(\pi_X\) y \(\pi_Y\)) para ver cuán diferentes son. La discordancia se mide contando cuántas veces un par de elementos está en un orden diferente en \(X\) en comparación con \(Y\).
   - Por ejemplo, si en \(X\) el segundo elemento viene antes que el primero pero en \(Y\) el primero viene antes que el segundo, eso es una discordancia.

5. **Calculo de la Distancia de Discordancia**:
   - La distancia de Kendall (\(\tau\)) es una forma común de medir esta discordancia, contando cuántas discordancias hay entre las posiciones de \(X\) e \(Y\).
   - Si hay muchas discordancias, significa que las listas están muy desalineadas.

6. **Normalización de la Distancia**:
   - Para obtener el coeficiente Xicorr, normalizas la cantidad de discordancias para que el valor resultante esté en un rango estándar (generalmente de -1 a 1).
   - La fórmula de normalización convierte la cantidad de discordancias en un valor que puede interpretarse como un coeficiente de correlación.

In [13]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xxlimited as xicor

# Función para calcular la matriz de correlación Xi
def calcular_matriz_xi(df):
    n = df.shape[1]
    matriz_xi = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i, n):
            if i == j:
                matriz_xi[i, j] = 1.0
            else:
                xi_valor = xicor.xicor(df.iloc[:, i], df.iloc[:, j])
                matriz_xi[i, j] = xi_valor
                matriz_xi[j, i] = xi_valor
    
    return pd.DataFrame(matriz_xi, index=df.columns, columns=df.columns)

# Calcular la matriz de correlación Xi
app_logger.info("Calculando la matriz de correlación Xi...")
matriz_xi = calcular_matriz_xi(train_data_filtrado)

# Visualizar la matriz de correlación
plt.figure(figsize=(20, 16))
sns.heatmap(matriz_xi, annot=False, cmap='coolwarm', vmin=0, vmax=1)
plt.title('Matriz de Correlación Xi')
plt.tight_layout()
plt.show()

app_logger.info("Se ha generado la matriz de correlación Xi y se ha visualizado como un mapa de calor.")

# Encontrar las correlaciones más fuertes
correlaciones_fuertes = matriz_xi.unstack().sort_values(ascending=False).drop_duplicates()
correlaciones_fuertes = correlaciones_fuertes[correlaciones_fuertes < 1.0]  # Excluir autocorrelaciones
top_correlaciones = correlaciones_fuertes.head(10)

print("Top 10 correlaciones más fuertes:")
print(top_correlaciones)
app_logger.info("Se han identificado las 10 correlaciones más fuertes utilizando el método Xi.")


2024-07-22 03:27:20,171 - INFO     - Calculando la matriz de correlación Xi...


AttributeError: module 'xxlimited' has no attribute 'xicor'

In [ ]:
# Características de porches y terrazas
app_logger.info("Creando características relacionadas con porches y terrazas...")

# Calcular el área total de porches
train_data['TotalPorchArea'] = train_data['OpenPorchSF'] + train_data['EnclosedPorch'] + \
                               train_data['3SsnPorch'] + train_data['ScreenPorch']

# Crear indicador binario para casas con porches
train_data['HasPorch'] = (train_data['TotalPorchArea'] > 0).astype(int)

app_logger.info("Se han creado nuevas características: 'TotalPorchArea' y 'HasPorch'.")



## New Possible Features
- Based on ``YearRemodAdd`` and ``YearBuilt`` as the dataset description stated, if they are equal that means that the house is not remodeled and if its different means that it has been remodeled, we can add a binary feature indicating this.
- Reduce ``YearBuilt`` and ``YrSold`` to ``TimeToSell``.
- Convert ``YearRemodAdd`` to ``TimeUntilRemod`` that means the time since it was built until it was remod, and ``RemodUntilSale`` that is the time since it was remod until it was sold.
- Porch and Deck Areas: Create a total porch area feature and a binary indicator for houses with porches.
- Proximity and Neighborhood Effects: Group neighborhoods into clusters based on median house prices to capture locality effects.
- GeoCode neighborhoods.
- Total Square Footage: Combine all square footage features (``1stFlrSF``, ``2ndFlrSF``, ``TotalBsmtSF``, etc.) into a single feature.

## Inconsistencies
- In some cases ``GarageYrBlt`` (year that the garage was built) was previous to ``YearBuilt`` which is not logical. We can modify this cases and transform those values to the year that the house was built assumming this criterion. This variable is related mostly with the built year and the Remodelation year that we can discard it as it only adds complexity with no info to the dataset.